In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.60.0

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [ ]:
# Function to reverse the order of characters in a string
def reverse(input_string: str) -> str:
    print("Calling reverse function")
    return input_string[::-1]

# Function to remove all spaces from a string
def remove_white_spaces(input_string: str) -> str:
    print("Calling remove_white_spaces function")
    return input_string.replace(" ", "")

In [ ]:
# Define FunctionDeclarations for the functions
reverse_function = FunctionDeclaration(
    name="reverse",
    description="Reverses a string",
    parameters={
        "type": "object",
        "properties": {
            "input_string": {
                "type": "string",
                "description": "The string to reverse"
            }
        }
    }
)

remove_white_spaces_function = FunctionDeclaration(
    name="remove_white_spaces",
    description="Removes spaces from a string",
    parameters={
        "type": "object",
        "properties": {
            "input_string": {
                "type": "string",
                "description": "The string to remove whitespaces"
            }
        }
    }
)

In [ ]:
# Define the tools with the functions we created
string_tool = Tool(
    function_declarations=[reverse_function, remove_white_spaces_function]
)

In [ ]:
# Initialize the model
model = GenerativeModel(
    model_name="gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    system_instruction=[
        "Fulfill the user's instructions.",
        "If asked to reverse a string or remove whitespace, call the provided functions.",
        "You may call one function after the other if needed.",
        "Repeat the result to the user.",
    ],
    tools=[string_tool],
)

# Initialize the chat model with tools and configuration
chat = model.start_chat()

In [ ]:
def handle_response(response):
    # Check if there's a function call in the response
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        # If no function call, print the response text
        print(response.text)
        return


    # If the function call is for the reverse_string function
    if function_call.name == "reverse":
        # Extract the input argument for the reverse function
        input_string = function_call.args["input_string"]

        # Call the reverse function
        reversed_string = reverse(input_string)  # assuming reverse_string is defined

        # Send the result back to the chat session
        response = chat.send_message(
            Part.from_function_response(
                name=function_call.name,
                response={
                    "content": reversed_string,
                },
            ),
        )
        # Handle the response recursively
        return handle_response(response)

    # If the function call is for the remove_white_spaces function
    elif function_call.name == "remove_white_spaces":
        # Extract the input argument for the remove_white_spaces function
        input_string = function_call.args["input_string"]

        # Call the remove_white_spaces function
        no_space_string = remove_white_spaces(input_string)  # assuming remove_white_spaces is defined

        # Send the result back to the chat session
        response = chat.send_message(
            Part.from_function_response(
                name=function_call.name,
                response={
                    "content": no_space_string,
                },
            ),
        )
        # Handle the response recursively
        return handle_response(response)

    else:
        # You shouldn't end up here
        print(function_call)

In [ ]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

Why don't scientists trust atoms? 

Because they make up everything! 



In [ ]:
response = chat.send_message("Reverse the following string: Hello world")
handle_response(response)

Calling reverse function
dlrow olleH 



In [ ]:
response = chat.send_message("Remove the white spaces from the following word: hello @ world. com")
handle_response(response)

Calling remove_white_spaces function
hello@world.com 



In [ ]:
response = chat.send_message("Reverse the following string and remove spaces: I do not understand the new sentence.")
handle_response(response)

Calling reverse function
Calling remove_white_spaces function
.ecnetneswenehtdnatsrednutonodI 



In [ ]:
response = chat.send_message("Tell me a fact. Then remove the spaces in the fact text and then reverse the fact text.")
print(response.candidates[0].content.parts[0].text)
handle_response(response)

A group of owls is called a parliament. 


Calling remove_white_spaces function
Calling reverse function
.tnemailrapadellacsislwofopuorgA 

